# Pipeline Completo: Clasificación de Buzón Institucional

Este notebook carga el archivo Excel original del buzón de quejas, sugerencias y felicitaciones, y aplica ambos modelos entrenados:

1. **Modelo de Intención**: Clasifica si es queja, sugerencia o felicitación (TF-IDF + Regresión Logística)
2. **Modelo de Departamento**: Asigna el departamento responsable (SetFit + FlashText + Anchoring)

**Salida**: CSV con comentario original, intención predicha y departamento asignado.

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import json
import re
import joblib
from pathlib import Path
from collections import defaultdict

# SetFit y Sentence Transformers para clasificación de departamentos
from setfit import SetFitModel
from sentence_transformers import SentenceTransformer

# FlashText para clasificación rápida por keywords
from flashtext import KeywordProcessor

# Métricas
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Librerías cargadas correctamente")

## 1. Carga del Archivo Original

Cargamos el Excel del buzón institucional y extraemos la columna de comentarios.

In [ ]:
# Rutas de archivos
RUTA_EXCEL = Path("clasificacion_intencion/Buzón de Quejas, Sugerencias y Felicitaciones (respuestas) (2).xlsx")

# Verificar que existe
if not RUTA_EXCEL.exists():
    raise FileNotFoundError(f"No se encontró el archivo: {RUTA_EXCEL}")

# Cargar Excel
df_original = pd.read_excel(RUTA_EXCEL)

print(f"📊 Archivo cargado: {len(df_original)} registros")
print(f"\n📋 Columnas disponibles:")
for col in df_original.columns:
    print(f"  • {col}")

In [ ]:
# Extraer solo la columna de comentarios
COLUMNA_COMENTARIOS = "Detalles de la queja, sugerencia o felicitación"

df = df_original[[COLUMNA_COMENTARIOS]].copy()
df = df.rename(columns={COLUMNA_COMENTARIOS: "comentario_original"})

# Eliminar filas sin comentario
df = df.dropna(subset=["comentario_original"])
df = df.reset_index(drop=True)

print(f"✅ {len(df)} comentarios válidos extraídos")
df.head()

## 2. Preprocesamiento Unificado

Aplicamos una limpieza de texto que funciona para ambos modelos:
- Convertir a minúsculas
- Eliminar saltos de línea y espacios múltiples
- Remover caracteres especiales (conservando acentos)
- Filtrar comentarios muy cortos (menos de 3 palabras)

In [ ]:
def limpiar_texto(texto):
    """
    Preprocesamiento unificado para ambos modelos.
    Conserva acentos y ñ para mejor clasificación en español.
    """
    if not isinstance(texto, str):
        texto = str(texto)
    
    texto = texto.lower()                          # Minúsculas
    texto = re.sub(r"\n", " ", texto)              # Saltos de línea → espacio
    texto = re.sub(r"\s+", " ", texto)             # Múltiples espacios → uno
    texto = re.sub(r"[^\wáéíóúñü\s]", "", texto)   # Quitar caracteres especiales
    texto = texto.strip()                          # Espacios al inicio/fin
    
    return texto

# Aplicar limpieza
df["comentario_limpio"] = df["comentario_original"].apply(limpiar_texto)

# Filtrar comentarios muy cortos (menos de 3 palabras)
df["n_palabras"] = df["comentario_limpio"].str.split().str.len()
df_filtrado = df[df["n_palabras"] >= 3].copy()

print(f"📝 Comentarios después de limpieza: {len(df_filtrado)}")
print(f"   (Se eliminaron {len(df) - len(df_filtrado)} comentarios muy cortos)")

# Mostrar ejemplo de limpieza
print("\n📋 Ejemplo de preprocesamiento:")
ejemplo_idx = 0
print(f"  Original: {df_filtrado.iloc[ejemplo_idx]['comentario_original'][:100]}...")
print(f"  Limpio:   {df_filtrado.iloc[ejemplo_idx]['comentario_limpio'][:100]}...")

## 3. Cargar Modelos Entrenados

Cargamos ambos modelos:
- **Intención**: TF-IDF vectorizer + Regresión Logística
- **Departamento**: SetFit + configuración de anclas

In [ ]:
# === MODELO DE INTENCIÓN (TF-IDF + LogisticRegression) ===
RUTA_VECTORIZADOR = Path("clasificacion_intencion/tfidf_vectorizer.pkl")
RUTA_MODELO_INTENCION = Path("clasificacion_intencion/clasificador.pkl")

vectorizador_tfidf = joblib.load(RUTA_VECTORIZADOR)
modelo_intencion = joblib.load(RUTA_MODELO_INTENCION)

print("✅ Modelo de Intención cargado")
print(f"   Clases: {modelo_intencion.classes_}")

In [ ]:
# === MODELO DE DEPARTAMENTO (SetFit) ===
RUTA_MODELO_DEPT = Path("models/clasificador_departamentos_setfit")
RUTA_CONFIG_DEPT = Path("models/clasificador_departamentos_config.json")

# Cargar modelo SetFit
modelo_departamento = SetFitModel.from_pretrained(RUTA_MODELO_DEPT)

# Cargar configuración (mapeos y anclas)
with open(RUTA_CONFIG_DEPT, "r", encoding="utf-8") as f:
    config_dept = json.load(f)

id2label_dept = config_dept["id2label"]
label2id_dept = config_dept["label2id"]
anclas_dept = config_dept.get("anclas", {})
UMBRAL_CONFIANZA = config_dept.get("umbral_confianza", 0.35)

print("✅ Modelo de Departamento cargado")
print(f"   Departamentos: {len(id2label_dept)}")
print(f"   Umbral de confianza: {UMBRAL_CONFIANZA}")

## 4. Configurar FlashText para Clasificación Rápida por Keywords

FlashText permite clasificación O(n) para casos obvios antes de usar el modelo SetFit.

In [ ]:
# Diccionario de palabras clave por departamento
KEYWORDS_POR_DEPARTAMENTO = {
    "Subdirección Administrativa": [
        "aire acondicionado", "clima", "luz", "electricidad", "mantenimiento",
        "limpieza", "baños", "mobiliario", "sillas", "escritorios", "focos",
        "instalaciones", "cafetería", "comedor", "estacionamiento"
    ],
    "Gestión Escolar": [
        "constancia", "certificado", "credencial", "inscripción", "reinscripción",
        "baja", "kardex", "calificaciones", "historial académico", "titulación",
        "servicio social", "prácticas profesionales", "boleta"
    ],
    "Unidad de Informática": [
        "internet", "wifi", "red", "computadora", "sistema", "plataforma",
        "correo institucional", "password", "contraseña", "software", "impresora",
        "laboratorio de cómputo", "proyector"
    ],
    "Subdirección Académica": [
        "maestro", "profesor", "docente", "clase", "horario", "materia",
        "evaluación docente", "plan de estudios", "tutorías", "asesorías"
    ],
    "Dirección": [
        "director", "director general", "rectoría", "política institucional"
    ]
}

# Crear procesador FlashText
keyword_processor = KeywordProcessor(case_sensitive=False)

for departamento, keywords in KEYWORDS_POR_DEPARTAMENTO.items():
    for kw in keywords:
        keyword_processor.add_keyword(kw, departamento)

def clasificar_por_keywords(texto):
    """
    Intenta clasificar usando palabras clave.
    Retorna (departamento, confianza) si encuentra match, sino (None, 0.0)
    """
    matches = keyword_processor.extract_keywords(texto.lower())
    if matches:
        freq = defaultdict(int)
        for dept in matches:
            freq[dept] += 1
        mejor_dept = max(freq, key=freq.get)
        return mejor_dept, 1.0
    return None, 0.0

print("✅ FlashText configurado con keywords para 5 departamentos")

## 5. Definir Funciones de Clasificación

Creamos las funciones que aplican cada modelo sobre los comentarios.

In [ ]:
def clasificar_intencion(texto):
    """
    Clasifica la intención del comentario: queja, sugerencia o felicitación.
    Retorna (clase, probabilidad_max)
    """
    X = vectorizador_tfidf.transform([texto])
    prediccion = modelo_intencion.predict(X)[0]
    probabilidades = modelo_intencion.predict_proba(X)[0]
    prob_max = max(probabilidades)
    
    return prediccion, prob_max


def clasificar_departamento(texto):
    """
    Clasifica el departamento responsable usando estrategia en cascada:
    1. FlashText (keywords)
    2. SetFit (modelo semántico)
    
    Retorna (departamento, metodo_usado)
    """
    # 1. Intentar con FlashText primero
    dept_kw, conf_kw = clasificar_por_keywords(texto)
    if dept_kw is not None:
        return dept_kw, "keywords"
    
    # 2. Usar SetFit
    prediccion = modelo_departamento.predict([texto])[0]
    
    # Convertir predicción a nombre de departamento
    if isinstance(prediccion, (int, np.integer)):
        departamento = id2label_dept.get(str(prediccion), "Desconocido")
    else:
        departamento = str(prediccion)
    
    return departamento, "setfit"


# Prueba rápida
texto_prueba = "El aire acondicionado del salón no funciona y hace mucho calor"
print(f"📝 Texto de prueba: '{texto_prueba}'")
print(f"\n🎯 Intención: {clasificar_intencion(texto_prueba)}")
print(f"🏛️ Departamento: {clasificar_departamento(texto_prueba)}")

## 6. Aplicar Clasificación a Todo el Dataset

Procesamos todos los comentarios con ambos modelos.

In [ ]:
from tqdm import tqdm

# Listas para almacenar resultados
intenciones = []
prob_intenciones = []
departamentos = []
metodos_dept = []

print("🔄 Clasificando comentarios...")
print(f"   Total a procesar: {len(df_filtrado)}\n")

# Procesar cada comentario
for idx, row in tqdm(df_filtrado.iterrows(), total=len(df_filtrado), desc="Clasificando"):
    texto = row["comentario_limpio"]
    
    # Clasificar intención
    intencion, prob = clasificar_intencion(texto)
    intenciones.append(intencion)
    prob_intenciones.append(prob)
    
    # Clasificar departamento
    depto, metodo = clasificar_departamento(texto)
    departamentos.append(depto)
    metodos_dept.append(metodo)

# Agregar columnas al dataframe
df_filtrado["intencion"] = intenciones
df_filtrado["prob_intencion"] = prob_intenciones
df_filtrado["departamento"] = departamentos
df_filtrado["metodo_clasificacion"] = metodos_dept

print("\n✅ Clasificación completada!")

## 7. Análisis de Resultados

Visualizamos la distribución de las clasificaciones.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de intenciones
ax1 = axes[0]
intencion_counts = df_filtrado["intencion"].value_counts()
colors_intencion = ["#ff6b6b", "#4ecdc4", "#45b7d1"]
bars1 = ax1.bar(intencion_counts.index, intencion_counts.values, color=colors_intencion)
ax1.set_title("📊 Distribución por Intención", fontsize=12, fontweight="bold")
ax1.set_xlabel("Intención")
ax1.set_ylabel("Cantidad")
for bar, val in zip(bars1, intencion_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, str(val), 
             ha='center', va='bottom', fontsize=10)

# Distribución de métodos de clasificación
ax2 = axes[1]
metodo_counts = df_filtrado["metodo_clasificacion"].value_counts()
colors_metodo = ["#96ceb4", "#ffeaa7"]
bars2 = ax2.bar(metodo_counts.index, metodo_counts.values, color=colors_metodo[:len(metodo_counts)])
ax2.set_title("🔧 Método de Clasificación de Departamento", fontsize=12, fontweight="bold")
ax2.set_xlabel("Método")
ax2.set_ylabel("Cantidad")
for bar, val in zip(bars2, metodo_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, str(val), 
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📈 Resumen de clasificación:")
print(f"\n🎯 Intenciones:")
for intent, count in intencion_counts.items():
    print(f"   {intent}: {count} ({count/len(df_filtrado)*100:.1f}%)")

print(f"\n🔧 Métodos usados:")
for metodo, count in metodo_counts.items():
    print(f"   {metodo}: {count} ({count/len(df_filtrado)*100:.1f}%)")

In [ ]:
# Distribución por departamento
plt.figure(figsize=(12, 6))
dept_counts = df_filtrado["departamento"].value_counts()

# Truncar nombres largos para visualización
dept_labels = [d[:25] + "..." if len(d) > 25 else d for d in dept_counts.index]

bars = plt.barh(dept_labels, dept_counts.values, color=plt.cm.viridis(np.linspace(0, 0.8, len(dept_counts))))
plt.xlabel("Cantidad de comentarios")
plt.title("🏛️ Distribución por Departamento", fontsize=12, fontweight="bold")

# Agregar valores
for bar, val in zip(bars, dept_counts.values):
    plt.text(val + 0.5, bar.get_y() + bar.get_height()/2, str(val), 
             va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n🏛️ Top 10 Departamentos:")
for dept, count in dept_counts.head(10).items():
    print(f"   {dept}: {count}")

## 8. Ejemplos de Clasificación

Mostramos algunos ejemplos para verificar la calidad de las predicciones.

In [ ]:
# Mostrar ejemplos aleatorios de cada intención
print("📋 Ejemplos de clasificación:\n")
print("=" * 100)

for intencion in ["queja", "sugerencia", "felicitación"]:
    subset = df_filtrado[df_filtrado["intencion"] == intencion]
    if len(subset) > 0:
        ejemplo = subset.sample(n=min(2, len(subset)), random_state=42)
        for _, row in ejemplo.iterrows():
            comentario = row["comentario_original"][:150]
            print(f"🎯 Intención: {row['intencion'].upper()}")
            print(f"🏛️ Departamento: {row['departamento']}")
            print(f"📝 Comentario: {comentario}{'...' if len(row['comentario_original']) > 150 else ''}")
            print(f"📊 Confianza intención: {row['prob_intencion']:.2%}")
            print(f"🔧 Método: {row['metodo_clasificacion']}")
            print("-" * 100)

## 9. Exportar Resultados a CSV

Guardamos el dataset procesado con las clasificaciones en la raíz del proyecto.

In [ ]:
# Preparar DataFrame de salida
df_salida = df_filtrado[[
    "comentario_original",
    "intencion",
    "departamento"
]].copy()

# Renombrar columnas para claridad
df_salida = df_salida.rename(columns={
    "comentario_original": "comentario"
})

# Guardar CSV en la raíz del proyecto
RUTA_SALIDA = Path("buzon_clasificado.csv")
df_salida.to_csv(RUTA_SALIDA, index=False, encoding="utf-8")

print(f"✅ Archivo guardado: {RUTA_SALIDA.absolute()}")
print(f"   Total de registros: {len(df_salida)}")
print(f"\n📋 Columnas en el archivo:")
for col in df_salida.columns:
    print(f"   • {col}")

# Mostrar preview
print(f"\n📄 Preview del archivo:")
df_salida.head(10)

## 10. Exportar Versión Detallada (Opcional)

También guardamos una versión con más información para análisis posterior.

In [ ]:
# Versión detallada con métricas adicionales
df_detallado = df_filtrado[[
    "comentario_original",
    "comentario_limpio",
    "intencion",
    "prob_intencion",
    "departamento",
    "metodo_clasificacion"
]].copy()

df_detallado = df_detallado.rename(columns={
    "comentario_original": "comentario",
    "comentario_limpio": "comentario_procesado",
    "prob_intencion": "confianza_intencion",
    "metodo_clasificacion": "metodo_departamento"
})

# Guardar versión detallada
RUTA_DETALLADO = Path("buzon_clasificado_detallado.csv")
df_detallado.to_csv(RUTA_DETALLADO, index=False, encoding="utf-8")

print(f"✅ Versión detallada guardada: {RUTA_DETALLADO.absolute()}")
print(f"\n📋 Columnas adicionales:")
print("   • comentario_procesado: texto limpio usado para clasificación")
print("   • confianza_intencion: probabilidad de la predicción de intención")
print("   • metodo_departamento: 'keywords' (FlashText) o 'setfit' (modelo)")

print("\n" + "=" * 60)
print("🎉 Pipeline completado exitosamente!")
print("=" * 60)
print(f"\n📁 Archivos generados en la raíz:")
print(f"   1. buzon_clasificado.csv - Versión simple (3 columnas)")
print(f"   2. buzon_clasificado_detallado.csv - Versión completa (6 columnas)")